In [ ]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

peft_model_id = "./kmonitor-gpt-fixed-loss-v2-46k"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(
    "NYTK/PULI-GPT-3SX", return_dict=True, load_in_8bit=True, device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained("NYTK/PULI-GPT-3SX")
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

model = PeftModel.from_pretrained(model, peft_model_id, device_map={'':0})

In [ ]:
from datasets import load_from_disk


def trunc_to(descr, n):
    if '.' in descr[int(n/2):n]:
        return descr[:descr[:n].rfind('.')+1]
    elif ' ' in descr[int(n/2):n]:
        return descr[:descr[:n].rfind(' ')+1]
    else:
        return descr[:n]


data = load_from_disk("./dataset-fixed")

# TODO better solution for labels and tokenization
test = [tokenizer('[klasszifikáció]\n' + trunc_to(samples["title"], 150) + '\n\n' + trunc_to(samples["description"], 700) + '\n\n' + trunc_to(samples['text'], 2500) + '\n\n###\n\ntéma:', return_tensors="pt") for samples in data['test']]
labels = []
for i, l in enumerate(test):
    if l:
        labels.append(data['test'][i]['theme'])

In [ ]:
from tqdm import tqdm
from transformers import GenerationConfig

tp = 0
tn = 0
fp = 0
fn = 0
invalid = 0

genconf = GenerationConfig(
    temperature=0
)

for n, d in tqdm(enumerate(test), total=3618):
    with torch.cuda.amp.autocast():
        d = d.to('cuda')
        output_tokens = model.generate(**d, max_new_tokens=2, pad_token_id=tokenizer.eos_token_id, generation_config=genconf)
        d = d.to('cpu')
        result = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
        r = result[result.rfind('téma:')+len('téma:'):]
        if len(r.strip()) > 0:
            r = r.strip()[0]

        if r != 'k' and r != 'e':
            invalid += 1
        if r == 'k':
            if labels[n] == 1:
                tp += 1
            else:
                fp += 1
        else:
            if labels[n] == 0:
                tn += 1
            else:
                fn += 1
        if n % 100 == 0:
            print(tp, fp, tn, fn)
            print((tp+tn)/(tp+fp+tn+fn))
print(tp, fp, tn, fn)
print((tp+tn)/(tp+fp+tn+fn))

In [ ]:
from tqdm import tqdm
from transformers import GenerationConfig

genconf = GenerationConfig(
    temperature=0
)

total = 0
for i in range(10):
    tp = 0
    tn = 0
    fp = 0
    fn = 0
    invalid = 0

    for n, d in tqdm(enumerate(test), total=3618):
        with torch.cuda.amp.autocast():
            d = d.to('cuda')
            output_tokens = model.generate(**d, max_new_tokens=2, pad_token_id=tokenizer.eos_token_id, generation_config=genconf)
            d = d.to('cpu')
            result = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
            r = result[result.rfind('téma:')+len('téma:'):]
            if len(r.strip()) > 0:
                r = r.strip()[0]

            if r != 'k' and r != 'e':
                invalid += 1
            if r == 'k':
                if labels[n] == 1:
                    tp += 1
                else:
                    fp += 1
            else:
                if labels[n] == 0:
                    tn += 1
                else:
                    fn += 1
    print(tp, fp, tn, fn)
    print((tp+tn)/(tp+fp+tn+fn))
    total += (tp+tn)/(tp+fp+tn+fn)
print(f'Test Accuracy: {total / 10: .5f}')